In [ ]:
import requests, os
from pathlib import Path

In [ ]:
current_path = Path(os.getenv('WORKING_DIR')) if os.getenv('WORKING_DIR') else Path().absolute()
current_path

In [ ]:
# load config
import configparser
config = configparser.ConfigParser()
config.read(current_path / 'config.ini')
assert 'secrets' in config.sections()

wh_url = config.get('secrets', 'slack_webhook_url')

In [ ]:
last_found = None
path = Path(current_path / 'last-community-resource-found.txt')
if path.exists():
    with path.open() as f:
        last_found = f.read()
last_found

In [ ]:
def send_message(msg):
    print(msg)
    r = requests.post(wh_url, json={'text': msg})
    assert r.text == 'ok'

In [ ]:
def notify_cr(cr):
    dataset_link = f"<{cr['dataset']['uri']}|{cr['dataset']['title']}>" if cr['dataset'] else '*None*'
    send_message(f":panda_face: new _CommunityResource_ on {dataset_link}: {cr['url']}")

In [ ]:
cr_url = 'https://www.data.gouv.fr/api/1/datasets/community_resources/?sort=-created_at'
r = requests.get(cr_url)
new_last_found = None
for cr in r.json()['data']:
    if last_found and last_found == cr['id']:
        break
    if not new_last_found:
        new_last_found = cr['id']
    notify_cr(cr)

In [ ]:
if new_last_found:
    with path.open('w') as f:
        f.write(new_last_found)
new_last_found